### 1. Starting a local MQTT broker

In [1]:
! mosquitto

1534908983: mosquitto version 1.4.15 (build date Sat, 07 Apr 2018 11:16:43 +0100) starting
1534908983: Using default config.
1534908983: Opening ipv4 listen socket on port 1883.
1534908983: Error: Address already in use


### 2. Starting the publishers

Both scripts [publisher-g1.py](./publisher-g1.py) and [publisher-uol.py](./publisher-uol.py) will retrieve news from [RSS / XML] (https://en.wikipedia.org/wiki/RSS) sources and publish them in the broker accordingly the following JSON structure:

```json
    {'title':   "Titulo da noticia",
     'summary': "Resumo da noticia",
     'link':    "http://...",
     'source':  "uol",
     'author':  "Asdrubal"
     }
```

In [2]:
import subprocess
publisher1 = subprocess.Popen(["python3","publisher-g1.py"])
publisher2 = subprocess.Popen(["python3","publisher-uol.py"])


#### **Suggestion:** open a terminal and listen to the /news/all topic in the broker. 

In terminal, enter:
```bash
mosquitto_sub -t "/news/all"
```

### 3. Connecting to the local MQTT broker

In [3]:
import paho.mqtt.client as paho
import json
from IPython.display import clear_output
from IPython.core.display import display, HTML

broker="localhost" 
port=1883

client1 = paho.Client("client1")                      #create client object

c = client1.connect(broker,port)                      #establishing the connection

### 4. Subscribing to the "/news/all" topic

In [4]:
pub_topic="/news/all"
status = client1.subscribe(("/news/all"))

### 5. Listening to all messages

In [6]:
def on_message(client, userdata, message):
    
    jsonObject = json.loads(str(message.payload.decode("utf-8")))
    
    print("msg.topic:     {0}"   .format(message.topic))
    print("news.tite:     {0}"   .format(jsonObject['title']))
    print("news.source:   {0}"   .format(jsonObject['source']))
    print("news.author:   {0}"   .format(jsonObject['author']))
    print("news.approved: {0}"   .format(jsonObject['approved']))
    display(HTML("""<a target="_blank" href="{0}">link</a>""".format(jsonObject['link'])))
    clear_output(wait=True)

    
client1.on_message = on_message
client1.loop_start()    #start the loop
clear_output(wait=True)

msg.topic:     /news/all
news.tite:     Quatro pessoas ficam feridas após táxi colidir com traseira de caminhão em rodovia no AM
news.source:   g1
news.author:   Luizadino
news.approved: False


### 6.1. Filtering by the _source_ field (g1)

In [10]:
def on_message(client, userdata, message):
    
    jsonObject = json.loads(str(message.payload.decode("utf-8")))
    
    if(jsonObject['source'] == "g1"):
        print("msg.topic:     {0}"   .format(message.topic))
        print("news.tite:     {0}"   .format(jsonObject['title']))
        print("news.source:   {0}"   .format(jsonObject['source']))
        print("news.author:   {0}"   .format(jsonObject['author']))
        print("news.approved: {0}"   .format(jsonObject['approved']))
        display(HTML("""<a target="_blank" href="{0}">link</a>""".format(jsonObject['link'])))
        clear_output(wait=True)

    
client1.on_message = on_message
client1.loop_start()    #start the loop
clear_output(wait=True)

msg.topic:     /news/all
news.tite:     Dois carros se envolvem em acidente em cruzamento de João Pessoa
news.source:   g1
news.author:   Yarapy
news.approved: True


### 6.2. Filtering by the _author_ field (Asdrubal)

In [11]:
def on_message(client, userdata, message):
    
    jsonObject = json.loads(str(message.payload.decode("utf-8")))
    
    if(jsonObject['author'] == "Asdrubal"):
        print("msg.topic:     {0}"   .format(message.topic))
        print("news.tite:     {0}"   .format(jsonObject['title']))
        print("news.source:   {0}"   .format(jsonObject['source']))
        print("news.author:   {0}"   .format(jsonObject['author']))
        print("news.approved: {0}"   .format(jsonObject['approved']))
        display(HTML("""<a target="_blank" href="{0}">link</a>""".format(jsonObject['link'])))
        clear_output(wait=True)

    
client1.on_message = on_message
client1.loop_start()    #start the loop
clear_output(wait=True)

msg.topic:     /news/all
news.tite:     Congresso de radiodifusão reúne empresários e diretores de emissoras
news.source:   g1
news.author:   Asdrubal
news.approved: False


### 6.3. Filtering by the _approved_ field 

In [3]:
# do it yourself